This script do a few things:
1. Find all the policies based on the existing bundles and list them 
2. Allow to delete all dependent bundles from a policy 
3. Allow to delete a policy 

In [23]:
import os
import requests
import json

In [56]:
# ----------------------------------------------------
#   ENV CONFIG & CONSTANTS
# ----------------------------------------------------
API_HOST = os.getenv("API_HOST", "https://domino.domino.tech")
API_KEY = os.getenv("API_KEY", "")
HEADERS = {
    "X-Domino-Api-Key": API_KEY,
    "Accept": "application/json"
}

In [57]:
#
#  Run this cell to identify all policies and get their IDs
#

url = f"{API_HOST}/api/governance/v1/bundles"

params = {"state": ["Active", "Archived"]}
resp = requests.get(url, headers=HEADERS, params=params)

if resp.status_code != 200:
    print(f"Error fetching bundles: {resp.status_code} - {resp.text}")
bundles = resp.json().get("data", [])

print (f"resp.status_code: {resp.status_code}")
print (f"resp: {resp}")
if bundles:
    print(f"Extracted bundles ({len(bundles)}).")
    # print(json.dumps(bundles, indent=2))
else:
    print("No bundles found.")

policies = {}
for b in bundles:
    policy_id = b.get("policyId")
    policy_name = b.get("policyName", "Unnamed Policy")
    if policy_id:
        policies[policy_id] = policy_name

if policies:
    print(f"Extracted Policies ({len(policies)}) from Bundles:")
    print(json.dumps(policies, indent=2))
else:
    print("No policies found.")

resp.status_code: 200
resp: <Response [200]>
Extracted bundles (66).
Extracted Policies (24) from Bundles:
{
  "be0fbb59-1807-4f40-bf2f-7707ad76671a": "Ethical AI Policy ",
  "74cf8a81-66ed-401d-8e2d-84bc3d90880c": "Business Risk Policy",
  "e26bb642-f04a-41df-aef3-5fae2eaa718e": "Model Development Policy v2",
  "b14b228d-0105-4817-9d5f-28d796116589": "ADaM Dataset",
  "75656cfa-70ca-4ad8-bdcd-a7ca1ae78c33": "Model Risk Management",
  "317d7fec-05e3-4ebf-abeb-25f06c658105": "Cyber Security Governance Policy",
  "d4de83c8-10ac-416c-a01e-bb2207edcd92": "Project Intake Questionnaire ",
  "8184778a-b882-4c6b-9a47-2d7f8fd03479": "z1",
  "52a6adff-8be2-451b-9236-a3cbfb16ca7d": "VisibilityRule",
  "057cd4ad-fa68-4e77-8d94-bf381cad196e": "Stage 1 Model Planning Process",
  "258642ce-5b4f-4b3c-88ce-d2d34bc68a93": "Decision Tree",
  "adaddb51-16e2-4bfb-86f5-ca298b2d2753": "Tony's Decision Tree",
  "22c77732-813f-446a-b088-91098c05245e": "NAIC",
  "4fd04487-b44c-4676-9c90-864241cb985d": "Business

In [68]:
# 
#
#  Run this cell to delete all bundles from a policy 
#

policy_to_delete = "a7773dd0-8e21-4edd-83cf-845491eb0e6e" # "0477ae9b-0a53-49a6-914a-3405e476cb42"

def delete_bundle(bundle_id):
    """
    Delete a bundle by its ID.
    """
    url = f"{API_HOST}/api/governance/v1/bundles/{bundle_id}"
    resp = requests.delete(url, headers=HEADERS)
    print(f"Delete bundle '{policy_id}'. Status code: {resp.status_code} - {resp.text}")
    return resp.status_code == 200

def delete_all_bundles(bundles):
    """
    Delete all bundles provided in the list and return a summary of the results.
    """
    results = {}
    for bundle in bundles:
        bundle_id = bundle.get("id")
        if bundle_id:
            success = delete_bundle(bundle_id)
            results[bundle_id] = success
    return results

def delete_bundles_for_policy(policy_id):
    # Fetch bundles for the given policy
    bundles = list_bundles(policy_id=policy_id)
    print(f"Found {len(bundles)} bundles under policy ID '{policy_id}'.")
    for bundle in bundles:
        # Print each bundle's details in a pretty format
        print(json.dumps(bundle, indent=2))
    
    if not bundles:
        print("No bundles found for this policy.")
        return

    # Delete all bundles under the policy
    results = delete_all_bundles(bundles)
    
    # Print deletion results for each bundle
    print("Deletion Results:")
    for bundle_id, success in results.items():
        status = "Success" if success else "Failed"
        print(f"Bundle ID {bundle_id}: {status}")

delete_bundles_for_policy(policy_to_delete)

Found 1 bundles under policy ID 'a7773dd0-8e21-4edd-83cf-845491eb0e6e'.
{
  "id": "a89c1efd-8b98-4b57-906e-2aa06ecfada9",
  "name": "Model 1.5",
  "createdAt": "2025-02-03T21:10:33.951657Z",
  "createdBy": {
    "firstName": "Ahmet",
    "id": "654d51dac89cb93b0f7a9b1f",
    "lastName": "Gyger",
    "userName": "ahmet_gyger"
  },
  "policyId": "a7773dd0-8e21-4edd-83cf-845491eb0e6e",
  "policyName": "Business Value Management 0.5",
  "projectId": "67a0d42b4c047941d18bc89d",
  "projectName": "GovernanceMQDemo2025",
  "projectOwner": "ahmet_gyger",
  "stage": "Define Business Value",
  "classificationValue": "",
  "state": "Active"
}
Delete bundle '75656cfa-70ca-4ad8-bdcd-a7ca1ae78c33'. Status code: 204 - 
Deletion Results:
Bundle ID a89c1efd-8b98-4b57-906e-2aa06ecfada9: Failed


In [69]:
# Delete the policy 

def delete_policy(policy_id):
    """
    Deletes a policy by its ID.

    Parameters:
      policy_id (str): The ID of the policy to delete.
    """
    url = f"{API_HOST}/api/governance/v1/policies/{policy_id}"
    resp = requests.delete(url, headers=HEADERS)
    
    if resp.status_code == 204:
        print(f"Policy '{policy_id}' deleted successfully.")
    else:
        print(f"Failed to delete policy '{policy_id}'. Status code: {resp.status_code} - {resp.text}")

# Example usage:
delete_policy(policy_to_delete)

Policy 'a7773dd0-8e21-4edd-83cf-845491eb0e6e' deleted successfully.
